In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
import pickle
import random

In [3]:
import seaborn as sns

In [4]:
import lightgbm as lgb

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [6]:
!pip install jupyter-dash

import plotly
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 6.8 MB/s 
     |████████████████████████████████| 1.6 MB 18.0 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


In [3]:
! pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from lazypredict.Supervised import LazyClassifier

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
!cp /content/gdrive/MyDrive/git_projects/plastic_extruder/plastic_extruder/extrusion.csv /content/

#Main data preparation steps from Extruder_a1.ipynb

In [7]:
df = pd.read_csv ('extrusion.csv')
df['date'] = pd.to_datetime(df['Datum'], format="%d.%m.%Y %H:%M")
df

,Datum,ST0_VARActAuftrag,ST110_VARExtr_0_HeizungZone_3_Regler_Y,ST110_VARExtr_0_HeizungZone_3_SollTemp,ST110_VARExtr_0_HeizungZone_4_Konfig,ST110_VARExtr_0_HeizungZone_4_Regler_X,ST110_VARExtr_0_HeizungZone_4_Regler_Y,ST110_VARExtr_0_HeizungZone_4_SollTemp,ST110_VARExtr_0_HeizungZone_5_Konfig,ST110_VARExtr_0_HeizungZone_5_Regler_X,...,ST114_VARTotalLen,ST114_VARTotalRolls,ST114_VARWdCloseCmd,ST114_VARWdSpeedHMI,ST114_VARWdSpSpeed,ST114_VARWdSpTens,ST114_VARWdTapeNum,ST114_VARWdTapeReductVal,ST114_VARWdTensVis,date
0,25.06.2018 4:40,0.00,9.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,...,3544103.00,2174.00,1.00,78.00,0.00,50.00,5.00,15.00,53.00,2018-06-25 04:40:00
1,25.06.2018 4:42,0.00,9.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,...,3544103.00,2174.00,1.00,81.00,0.00,50.00,5.00,15.00,49.00,2018-06-25 04:42:00
2,25.06.2018 4:45,0.00,9.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,...,3544103.00,2174.00,1.00,81.00,0.00,50.00,5.00,15.00,48.00,2018-06-25 04:45:00
3,25.06.2018 4:47,0.00,10.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,...,3544103.00,2174.00,1.00,81.00,0.00,50.00,5.00,15.00,50.00,2018-06-25 04:47:00
4,25.06.2018 4:49,0.00,11.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,...,3544103.00,2174.00,1.00,81.00,0.00,50.00,5.00,15.00,51.00,2018-06-25 04:49:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226531,25.06.2019 6:36,0.00,2.00,165.00,1.00,170.00,0.00,165.00,1.00,165.00,...,799185.00,387.00,1.00,57.00,0.00,55.00,5.00,15.00,51.00,2019-06-25 06:36:00
226532,25.06.2019 6:38,0.00,2.00,165.00,1.00,170.00,0.00,165.00,1.00,165.00,...,799185.00,387.00,1.00,56.00,0.00,55.00,5.00,15.00,51.00,2019-06-25 06:38:00
226533,25.06.2019 6:40,0.00,2.00,165.00,1.00,170.00,0.00,165.00,1.00,165.00,...,799185.00,387.00,1.00,57.00,0.00,55.00,5.00,15.00,51.00,2019-06-25 06:40:00
226534,25.06.2019 6:42,0.00,2.00,165.00,1.00,170.00,0.00,165.00,1.00,165.00,...,799185.00,387.00,1.00,56.00,0.00,55.00,5.00,15.00,51.00,2019-06-25 06:42:00


In [8]:
df['classified'] = np.where(df['ST110_VAREx_0_SDickeIst']== 0, 0, 1)
df['classified']

0         1
1         1
2         1
3         1
4         1
         ..
226531    1
226532    1
226533    1
226534    1
226535    1
Name: classified, Length: 226536, dtype: int64

In [9]:
classified_list = df['classified'].to_list()

In [10]:
#remove repeating adjacent zero when machine is not working by adding a new column
classified_list2 = []
count = 0
for value in classified_list:
    if count == 0:
        classified_list2.append(1)
    else:
        if classified_list[count] == 0:
            count2 = count-1
            if classified_list[count2] == 0:
                classified_list2.append(0)
            else:
                classified_list2.append(1)
        else:
            classified_list2.append(1)
    count = count+1

In [11]:
#remove repeating adjacent zero when machine is not working by adding a new column
classified_list2 = []
count = 0
for value in classified_list:
    if count == 0:
        classified_list2.append(1)
    else:
        if classified_list[count] == 0:
            count2 = count-1
            if classified_list[count2] == 0:
                classified_list2.append(0)
            else:
                classified_list2.append(1)
        else:
            classified_list2.append(1)
    count = count+1

In [12]:
df['classified2'] = classified_list2

In [13]:
df2 = df[df['classified2'] == 1] #Include only first zero values in case of continuous zero thickness

## Positions where failure happends

In [14]:
 df2_re = df2.reset_index(drop=True) #reindexing as only first row with zero thickness included

In [15]:
# to know row number where classified is 0 after reindexing
row_list1_re= df2_re[df2_re['classified'] == 0].index.to_numpy()
row_list1_re

array([   183,    382,    614,    623,    625,    630,    635,    637,
          647,   1681,   1685,   1688,   1817,   1833,   1887,   2516,
         2770,   3781,   3872,   3914,   4017,   5524,   5527,   5808,
         6374,   6420,   6994,   7870,   8149,   8826,   8903,   8931,
         9487,   9590,   9690,   9961,  10025,  10044,  10427,  10850,
        10875,  11014,  11063,  11088,  11438,  11789,  12083,  12095,
        12120,  12608,  12610,  12640,  12649,  12831,  12839,  12866,
        12978,  13001,  13011,  13157,  13166,  13425,  13458,  14076,
        14538,  14582,  14591,  14601,  14685,  15458,  15462,  16514,
        16518,  17006,  17008,  17033,  17037,  17048,  17068,  17070,
        17073,  17092,  17095,  17101,  17116,  17136,  17480,  18283,
        18290,  18618,  18831,  18833,  18840,  18849,  19074,  19090,
        19107,  19109,  19417,  19430,  19550,  19689,  19717,  19789,
        19810,  19812,  19953,  20092,  20101,  20249,  20415,  20472,
      

In [16]:
# Removing multiple values from a single event by removing closer failure event
c1 = 1
row_list1_re_f1 = []

while c1 < len(row_list1_re):
  c2 = c1 - 1 
  if c1 == 1:
    row_list1_re_f1.append(row_list1_re[0])      # adding the first value

  n1 = row_list1_re[c1]
  n2 = row_list1_re[c2]
  d = abs(n2-n1)
  if d > 20:
    row_list1_re_f1.append(row_list1_re[c1])
  c1 = c1+1

In [17]:
# Removing failures due to dimension changes
row_list1_re2_f2 = []
for tpoint in row_list1_re_f1:
  n1 = tpoint - 100
  n2 = tpoint - 50
  n3 = tpoint + 100
  n4 = tpoint + 150
  values_before = df2_re.ST110_VAREx_0_SDickeIst[n1:n2]
  values_after = df2_re.ST110_VAREx_0_SDickeIst[n3:n4]
  avg1 = values_before.mean()
  avg2 = values_after.mean()
  d_avg = abs(avg1-avg2)
  if d_avg < 2:
    row_list1_re2_f2.append(tpoint)

In [21]:
len(row_list1_re2_f2)

222

In [22]:
# generate sequuences without stoppage for machine learning comparison
#make same number random numbers 
row_list2=[]
for i in range(2000):
  r=random.randint(1,197110)
  if r not in row_list2: row_list2.append(r)

In [23]:
# generate sequuences without stoppage for machine learning comparison
#make same number random numbers 
row_list2=[]
for i in range(2000):
  r=random.randint(1,197110)
  if r not in row_list2: row_list2.append(r)

In [24]:
#check any zero values in randomly selected points
zero_val = []
for numb in row_list2:
  i=0
  while i<20:
    numb2 = numb-i
    if df2_re.ST110_VAREx_0_SDickeIst[numb2] == 0:
      if numb not in zero_val: zero_val.append(numb)
      #zero_val.append(numb)
      #row_list2.remove(numb)

    i = i+1     

In [25]:
row_list2_f1 = list(set(row_list2) - set(zero_val))

In [26]:
#convert type numpy.int64 to int for combining
row_list1_re2_f2_conv = []
for values in row_list1_re2_f2:
  pyval = values.item()
  row_list1_re2_f2_conv.append(pyval)
type(row_list1_re2_f2_conv[10])

int

In [27]:
row_list_comb= []
row_list_comb = row_list2_f1 + row_list1_re2_f2_conv
row_list_comb.sort()
row_list_comb[:10]

[26, 183, 239, 320, 333, 407, 418, 584, 588, 743]

## Preparing data sequences for machine learning

In [28]:
df5 = df2_re.copy()
df5a = df5.copy()  #storing classified value
df5 = df5.drop(columns=['Datum', 'classified','classified2','date','ST110_VAREx_0_SDickeIst' ])

In [29]:
#make sequence 

y_target = []

i=0

for row_n in row_list_comb:
    row_number = row_n-10
    dfa1 = df5.loc[[row_number]]
    dfa2 = df5.loc[[row_number+1]]
    dfa3 = df5.loc[[row_number+2]]
    dfa4 = df5.loc[[row_number+3]]
    dfa5 = df5.loc[[row_number+4]]
    dfa6= df5.loc[[row_number+5]]
    dfa7 = df5.loc[[row_number+6]]
    dfa8= df5.loc[[row_number+7]]
    dfa9= df5.loc[[row_number+8]]
    dfa10= df5.loc[[row_number+9]]
    dfa1 = dfa1.reset_index(drop=True)
    dfa2 = dfa2.reset_index(drop=True)
    dfa3 = dfa3.reset_index(drop=True)
    dfa4 = dfa4.reset_index(drop=True)
    dfa5 = dfa5.reset_index(drop=True)
    dfa6 = dfa6.reset_index(drop=True)
    dfa7 = dfa7.reset_index(drop=True)
    dfa8 = dfa8.reset_index(drop=True)
    dfa9 = dfa9.reset_index(drop=True)
    #dfa10 = dfa10.reset_index(drop=True) # removed as it for predict this point
    
    dfa11 = pd.concat([dfa1,dfa2,dfa3,dfa4,dfa5,dfa6,dfa7,dfa8,dfa9], axis=1)
    dfa11 = dfa11.reset_index(drop=True)
    dfa12 = dfa11.loc[[0]]
    if i == 0:
        df7 = dfa12
    else:
        df7 = pd.concat([df7, dfa12], axis=0)
    y_value = df5a.loc[row_number+10]['classified']
    y_target.append(y_value)
    i = i+1


In [30]:
df7

,ST0_VARActAuftrag,ST110_VARExtr_0_HeizungZone_3_Regler_Y,ST110_VARExtr_0_HeizungZone_3_SollTemp,ST110_VARExtr_0_HeizungZone_4_Konfig,ST110_VARExtr_0_HeizungZone_4_Regler_X,ST110_VARExtr_0_HeizungZone_4_Regler_Y,ST110_VARExtr_0_HeizungZone_4_SollTemp,ST110_VARExtr_0_HeizungZone_5_Konfig,ST110_VARExtr_0_HeizungZone_5_Regler_X,ST110_VARExtr_0_HeizungZone_5_Regler_Y,...,ST114_VARTensPlusSp,ST114_VARTotalLen,ST114_VARTotalRolls,ST114_VARWdCloseCmd,ST114_VARWdSpeedHMI,ST114_VARWdSpSpeed,ST114_VARWdSpTens,ST114_VARWdTapeNum,ST114_VARWdTapeReductVal,ST114_VARWdTensVis
0,0.00,8.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,23.00,...,21.00,3548567.00,2177.00,1.00,82.00,0.00,53.00,5.00,15.00,51.00
0,0.00,10.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,22.00,...,21.00,3576430.00,2196.00,1.00,82.00,0.00,53.00,5.00,15.00,53.00
0,0.00,9.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,22.00,...,21.00,3583654.00,2202.00,1.00,81.00,0.00,53.00,5.00,15.00,52.00
0,0.00,9.00,190.00,1.00,194.00,0.00,190.00,1.00,190.00,21.00,...,21.00,3597018.00,2211.00,1.00,81.00,0.00,53.00,5.00,15.00,53.00
0,0.00,9.00,190.00,1.00,194.00,0.00,190.00,1.00,190.00,20.00,...,21.00,3599988.00,2213.00,1.00,81.00,0.00,53.00,5.00,15.00,53.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.00,0.00,175.00,1.00,194.00,0.00,180.00,1.00,182.00,0.00,...,22.00,751274.00,364.00,1.00,28.00,0.00,100.00,5.00,15.00,129.00
0,0.00,0.00,173.00,1.00,191.00,0.00,173.00,1.00,187.00,0.00,...,22.00,754311.00,370.00,1.00,24.00,0.00,140.00,5.00,15.00,118.00
0,0.00,0.00,176.00,1.00,192.00,0.00,176.00,1.00,189.00,0.00,...,22.00,758959.00,373.00,1.00,25.00,0.00,140.00,5.00,15.00,134.00
0,0.00,0.00,176.00,1.00,192.00,0.00,176.00,1.00,190.00,0.00,...,22.00,762047.00,375.00,1.00,24.00,0.00,140.00,5.00,15.00,140.00


In [31]:
len(y_target)

2064

In [32]:
y_target[:10]

[1, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [33]:
df8 = df7.copy()

In [34]:
df8 = df8.reset_index(drop=True)
df8

,ST0_VARActAuftrag,ST110_VARExtr_0_HeizungZone_3_Regler_Y,ST110_VARExtr_0_HeizungZone_3_SollTemp,ST110_VARExtr_0_HeizungZone_4_Konfig,ST110_VARExtr_0_HeizungZone_4_Regler_X,ST110_VARExtr_0_HeizungZone_4_Regler_Y,ST110_VARExtr_0_HeizungZone_4_SollTemp,ST110_VARExtr_0_HeizungZone_5_Konfig,ST110_VARExtr_0_HeizungZone_5_Regler_X,ST110_VARExtr_0_HeizungZone_5_Regler_Y,...,ST114_VARTensPlusSp,ST114_VARTotalLen,ST114_VARTotalRolls,ST114_VARWdCloseCmd,ST114_VARWdSpeedHMI,ST114_VARWdSpSpeed,ST114_VARWdSpTens,ST114_VARWdTapeNum,ST114_VARWdTapeReductVal,ST114_VARWdTensVis
0,0.00,8.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,23.00,...,21.00,3548567.00,2177.00,1.00,82.00,0.00,53.00,5.00,15.00,51.00
1,0.00,10.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,22.00,...,21.00,3576430.00,2196.00,1.00,82.00,0.00,53.00,5.00,15.00,53.00
2,0.00,9.00,190.00,1.00,193.00,0.00,190.00,1.00,190.00,22.00,...,21.00,3583654.00,2202.00,1.00,81.00,0.00,53.00,5.00,15.00,52.00
3,0.00,9.00,190.00,1.00,194.00,0.00,190.00,1.00,190.00,21.00,...,21.00,3597018.00,2211.00,1.00,81.00,0.00,53.00,5.00,15.00,53.00
4,0.00,9.00,190.00,1.00,194.00,0.00,190.00,1.00,190.00,20.00,...,21.00,3599988.00,2213.00,1.00,81.00,0.00,53.00,5.00,15.00,53.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,0.00,0.00,175.00,1.00,194.00,0.00,180.00,1.00,182.00,0.00,...,22.00,751274.00,364.00,1.00,28.00,0.00,100.00,5.00,15.00,129.00
2060,0.00,0.00,173.00,1.00,191.00,0.00,173.00,1.00,187.00,0.00,...,22.00,754311.00,370.00,1.00,24.00,0.00,140.00,5.00,15.00,118.00
2061,0.00,0.00,176.00,1.00,192.00,0.00,176.00,1.00,189.00,0.00,...,22.00,758959.00,373.00,1.00,25.00,0.00,140.00,5.00,15.00,134.00
2062,0.00,0.00,176.00,1.00,192.00,0.00,176.00,1.00,190.00,0.00,...,22.00,762047.00,375.00,1.00,24.00,0.00,140.00,5.00,15.00,140.00


In [35]:
X = df8.fillna(0)
y = y_target

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [39]:
X_train_num = X_train.to_numpy()
X_test_num = X_test.to_numpy()

In [37]:
#LazyClassifier models
clf_mod = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)
### fitting data in LazyClassifier
models, predictions = clf_mod.fit(X_train, X_test, y_train, y_test)
### lets check which model did better on rubber Dataset
print(models)

100%|██████████| 29/29 [00:00<00:00, 428.66it/s]

Empty DataFrame
Columns: [Accuracy, Balanced Accuracy, ROC AUC, F1 Score, Time Taken]
Index: []


In [40]:
#LazyClassifier models
clf_mod = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)
### fitting data in LazyClassifier
models, predictions = clf_mod.fit(X_train_num, X_test_num, y_train, y_test)
### lets check which model did better on rubber Dataset
print(models)

100%|██████████| 29/29 [02:50<00:00,  5.90s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
Perceptron                         0.78               0.63     0.63      0.81   
LinearSVC                          0.83               0.61     0.61      0.84   
GaussianNB                         0.59               0.60     0.60      0.67   
PassiveAggressiveClassifier        0.83               0.60     0.60      0.84   
BernoulliNB                        0.47               0.59     0.59      0.56   
NearestCentroid                    0.41               0.58     0.58      0.50   
LogisticRegression                 0.88               0.57     0.57      0.87   
SGDClassifier                      0.87               0.56     0.56      0.86   
RidgeClassifier                    0.87               0.56     0.56      0.86   
LabelPropagation                   0.24               0.55     0.55      0.28   
LabelSpreading              